In [1]:
import socketio
import torch as th
from torch import nn
import syft as sy
import numpy as np
import binascii

In [2]:
import asyncio
from syft.workers import BaseWorker

class SioWorker(BaseWorker):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.sio = socketio.Client()
        self.sio.connect('http://localhost:5000')

    def _send_msg(self, message: bin, location: BaseWorker) -> bin:
        return location._recv_msg(message)
    
    def _recv_msg(self, message: bin) -> bin:
        self.sio.emit('message', message) # Block and wait for the response
        return sy.serde.serialize(b'')


In [3]:
hook = sy.TorchHook(th)

In [4]:
# Use Numpy serialization strategy
sy.serde._serialize_tensor = sy.serde.numpy_tensor_serializer
sy.serde._deserialize_tensor = sy.serde.numpy_tensor_deserializer
sy.serde._apply_compress_scheme = sy.serde.apply_no_compression

In [5]:
phone = SioWorker(hook, id="phone", verbose=True)

In [7]:
x = th.tensor([10,20,30,40,50.])
x_ptr = x.send(phone)

In [8]:
y = th.tensor([20,20,20,20,20.])
y_ptr = y.send(phone)

In [9]:
z = x_ptr + y_ptr

In [ ]:
print(z)
print(z.get())

In [ ]:
print(x_ptr.id_at_location)

In [ ]:
server_id = None
@sio.on('client_registered')
def on_client_registered(args):
    server_id = args

In [ ]:
sio.connect('http://localhost:5000')

In [ ]:
# Use Numpy serialization strategy
sy.serde._serialize_tensor = sy.serde.numpy_tensor_serializer
sy.serde._deserialize_tensor = sy.serde.numpy_tensor_deserializer
sy.serde._apply_compress_scheme = sy.serde.apply_no_compression

In [ ]:
# Simple send of a message containing a tensor. This is not how PySyft wraps the message!
# data = th.tensor([[0, 1.]], requires_grad=True)
# bin_data = sy.serde.serialize(data)
# sio.emit('message', bin_data)

In [ ]:
# This is PySyft message when doing x.send(bob) except the owner stuff
data = th.tensor([[0, 1.]], requires_grad=True)
pysyft_message = (sy.codes.MSGTYPE.OBJ, data)
bin_message = sy.serde.serialize(pysyft_message)
print(bin_message)
sio.emit('message', bin_message)

In [ ]:
# We can now add two tensors in Android (pointers are coming soon)
x = th.tensor([[0, 1.]])
y = th.tensor([[1., 1.]])
operation = ("__add__", (x, y))
pysyft_message = (sy.codes.MSGTYPE.CMD, operation)
bin_message = sy.serde.serialize(pysyft_message)
print(bin_message)
sio.emit('message', bin_message)

In [ ]:
# Adding two pointers. I am faking here the worker to get the info we need
bob = sy.VirtualWorker(hook, id="bob")
x = th.tensor([[0, 1.]])
y = th.tensor([[1., 1.]])
x_ptr = x.send(bob)
y_ptr = y.send(bob)

In [ ]:
z = x_ptr + y_ptr

In [ ]:
# Send through the socket and let Android read it (but it was for Bob!!)


In [ ]:
sio.disconnect()